Text-to-SQL Project: Prompt Generation & SQL Querying

This notebook handles the **core logic of generating SQL queries** from natural language questions using the loaded SQLite database of Eagles statistics.

Workflow Diagram:
User Input
   │
   ▼
[Step 1] → classify intent + extract entities via LLM
   │
   ▼
[Step 2] → build SQL prompt using schema + examples
   │
   ▼
[Step 3] → generate SQL via LLM
   │
   ▼
[Step 5] → execute SQL on SQLite
   │
   ▼
[Step 6] → optional agentic reflection + fix SQL via LLM
   │
   ▼
[Step 7] → execute SQL on SQLite (if regenerated from previous step)
   │
   ▼
[Step 8] → return final answer via LLM


STEP 1: Intent & Entity Classification

This step interprets the user’s natural language question to determine what kind of data they’re asking for and which details are relevant. This will be used to 

🔍 Purpose:

Turn a raw question like:
"How many rushing yards did Jalen Hurts have against Dallas in 2024?"
into structured metadata like:
{
  "intent": "player_week",
  "entities": {
    "season": 2024,
    "week": null,
    "player": "Jalen Hurts",
    "opponent": "DAL"
  }
}

This metadata will be used to create the SQL statement.

WORKFLOW:
1.	Define Supported Intents and Entities 
    a.	The system supports four kinds of questions:
        i.	player_season: full-season stats for a player
        ii.	player_week: opponent-specific or single-week stats for a player
        iii. team_game: performance in a specific game/week
        iv.	team_season: team-wide totals for an entire season
    b. From the user question identify entities that can be passed to the LLM to help generate the SQL statement: season, week, player, opponent.
2.	Build the Prompt for OpenAI
    a.	Creates a detailed system prompt that:
        i.	Describes the 4 intent types
        ii.	Asks for extraction of fields: season, week, player, opponent
        iii.	Provides rules, like never treating "PHI" as the opponent
        iv.	Requires valid JSON output only — no explanations
3.	Call the OpenAI Model
    a.	Sends the prompt to the model (e.g., GPT-4o-mini) with strict formatting
    b.	Expects a structured JSON response
4.	Parse and Clean the Model Output
    a.	Removes any extra formatting (like ```json code blocks)
    b.	Converts the output string into a Python dictionary
    c.	Validates opponent (must be one of 32 NFL teams)
    d.	Fills in missing fields with null if needed
5.	Handle Errors Gracefully
    a.	If parsing fails or response is malformed:
    b.	Default to intent = "team_season"
    c.	Return empty entities (None)



In [2]:
import os
import json
import httpx
from openai import OpenAI
from dotenv import load_dotenv
import re

#set cwd to parent dir
os.chdir("..")

# -------------------------------
# Load environment
# -------------------------------
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# -------------------------------
# Initialize OpenAI Client
# -------------------------------
client = OpenAI(
    api_key=OPENAI_API_KEY,
    http_client=httpx.Client(verify=os.environ.get("REQUESTS_CA_BUNDLE"))
)

# -------------------------------
# NFL Teams
# -------------------------------
NFL_TEAMS = [
    "ARI","ATL","BAL","BUF","CAR","CHI","CIN","CLE","DAL","DEN",
    "DET","GB","HOU","IND","JAX","KC","LV","LAC","LAR","MIA",
    "MIN","NE","NO","NYG","NYJ","PHI","PIT","SEA","SF","TB",
    "TEN","WAS"
]

team_list = ", ".join(NFL_TEAMS)
# -------------------------------
# Helper: Normalize CJ → C.J.
# -------------------------------
def normalize_player_name(name: str) -> str:
    if name is None:
        return None

    parts = name.split()
    if len(parts) > 1 and len(parts[0]) in {2, 3} and parts[0].isupper():
        # Convert AJ → A.J.   CJ → C.J.
        initials = ".".join(parts[0]) + "."
        return f"{initials} {' '.join(parts[1:])}"

    return name

# -------------------------------
# Intent + Entity Classifier
# -------------------------------
def classify_intent_and_extract_entities(question: str) -> dict:

    prompt = f"""
You are a classifier and entity extractor for a sports assistant focused on the Philadelphia Eagles.

You MUST return JSON in exactly the following structure:
{{
  "intent": "player_season | player_week | team_game | team_season",
  "entities": {{
    "season": integer or null,
    "week": integer or null,
    "player": string or null,
    "opponent": string or null
  }}
}}

Do NOT return JSON that is missing the "intent" or "entities" fields.

---

Your job is to:
1. Classify the question into one of these intents:
   - "player_season": full-season stats for a specific player (e.g. total TDs in 2023)
   - "player_week": weekly or opponent-specific stats for a player (e.g. vs Dallas or Week 9)
   - "team_game": team performance in a specific game or week
   - "team_season": team-wide totals for an entire season

2. Extract the following fields inside the "entities" object:
   - "season": integer (e.g. 2023) or null
   - "week": integer (e.g. 9) or null
   - "player": string (e.g. "Jalen Hurts" or "A.J. Brown") or null
   - "opponent": NFL team code (e.g. "DAL") or null

Guidelines:
- PHI should never be extracted as the opponent — the team is always the Eagles.
- Only extract opponents that are in this list: {team_list}
- Preserve initials in names like "A.J. Brown", "C.J. Gardner-Johnson"
- Return valid JSON only — no explanations, no formatting, no markdown fences.

User question:
{question}
"""


    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=200
        )

        raw = response.choices[0].message.content
        #print("RAW MODEL OUTPUT:", repr(raw))

        # Strip code fences if present
        if raw.startswith("```"):
            raw = re.sub(r"```(json)?", "", raw).strip("` \n")

        # Parse JSON
        result = json.loads(raw)

        # ----------------------------------------------------
        # 🔥 FIX: Convert flat structure → nested structure
        # ----------------------------------------------------
        if "entities" not in result:
            result = {
                "intent": result.get("intent", "team_season"),
                "entities": {
                    "season": result.get("season"),
                    "week": result.get("week"),
                    "player": result.get("player"),
                    "opponent": result.get("opponent")
                }
            }

        entities = result["entities"]

        # ----------------------------------------------------
        # Normalize opponent field
        # ----------------------------------------------------
        opp = entities.get("opponent")
        if opp:
            opp = opp.upper()
            entities["opponent"] = opp if opp in NFL_TEAMS else None
        else:
            entities["opponent"] = None

        # ----------------------------------------------------
        # Normalize player name (AJ → A.J.)
        # ----------------------------------------------------
        entities["player"] = normalize_player_name(entities.get("player"))

        # ----------------------------------------------------
        # Validate intent
        # ----------------------------------------------------
        valid_intents = {"player_week", "player_season", "team_game", "team_season"}
        if result.get("intent") not in valid_intents:
            result["intent"] = "team_season"

        # Return final normalized result
        return {
            "intent": result["intent"],
            "entities": entities
        }

    except Exception as e:
        print("JSON PARSE ERROR:", e)
        return {
            "intent": "team_season",
            "entities": {
                "season": None,
                "week": None,
                "player": None,
                "opponent": None
            }
        }


STEP 2: SQL Builder

This step transforms the classified intent and extracted entities into a structured SQL query that can be executed on our SQLite database.

PURPOSE: Turn metadata like:
{
  "intent": "player_week",
  "entities": {
    "season": 2024,
    "week": null,
    "player": "Jalen Hurts",
    "opponent": "DAL"
  }
}
 Into a valid SQL query like:

SELECT SUM(rushing_yards) AS total_rushing_yards
FROM player_week
WHERE player_display_name = 'Jalen Hurts'
  AND opponent_team = 'DAL'
  AND season = 2024;

This SQL is later executed against the SQLite database to retrieve the final answer.

WORKFLOW
1. Select the Table

The system maps the classified intent to the correct table:

| Intent Type     | Meaning                                     | Table Used    |
| --------------- | ------------------------------------------- | ------------- |
| `player_week`   | Player stats by week or against an opponent | `player_week` |
| `player_season` | Full-season player totals                   | `players`     |
| `team_game`     | Team performance in a specific game/week    | `games`       |
| `team_season`   | Full-season team totals                     | `seasons`     |

2. Retrieve the Table Schema
The builder uses:PRAGMA table_info(<table_name>);
This retrieves:

column names
column types
primary keys

The schema is passed to the LLM so the model knows exactly what fields exist and avoids hallucinating columns.

3. Build the Prompt for the LLM

The SQL Builder constructs a natural-language prompt containing:

    1.The table name
    2.The full schema
    3.The user’s question
    4.Rules for safe SQL generation:
        a.Use only valid SQLite syntax
        b.Filter on:
            season
            week
            player_display_name
            opponent_team
        c.Avoid filtering by PHI (every row is already Eagles data)
        d.Aggregate metrics (e.g., SUM) if multiple rows are implied.
        
This gives the model all context needed to generate correct SQL.

4. Generate SQL Using OpenAI: The builder calls the LLM to produce SQL
5. Clean and Format SQL

In [ ]:
import sqlite3
import re

#--------------------------------
#SQL Rules
#--------------------------------


def sql_rules(team_list: str) -> str:
    return (
            "- Use ONLY the extracted entities above when building the WHERE clause.\n"
            "- NEVER pull team names or player names from the natural language question.\n"
            "- Use player_display_name for player filters.\n"
            "- Do NOT filter by team since all teams are PHI\n" 
            "- If the user asks for total 'yards' for a player, include both `passing_yards` and `rushing_yards` if available.\n"
            "- If the user asks about a change over be sure to include relevant time fields like year or week\n"
            "- If including filters for position, only use standard abbreviations (e.g., 'QB', 'WR', 'RB').\n"
            "- Do NOT include any field in the WHERE clause that is not listed in the table schema.\n"
            f"- Only extract opponents that are in this list: {team_list}"
            "- if per game stats are requested for a season, remember there are per game stats available in the games and seasons tables. They will have per_game in the field name.\n"
            "- Only the games table has an opponent field. If opponent level aggregation is needed it must be done in the games table.\n"
            "- In the games table, the win and loss fields are boolean TRUE/FALSE values.\n"
            "- If stats are requested for a player in a specific game, use the player_week table.\n"
            "- If stats are requested for a player in a specific season, use the player_season table.\n"
            "- When querying the players table, season_type field should be 'REG+POST' unless regular-season(REG) or post-season(POST) stats are specifically requested.\n"
            "- If a question regarding when an event happened (e.g., highest yards in a game), be sure to include as year and week fields to identify the specific game.\n"
            )

sql_rules_text = sql_rules(team_list)

# -------------------------------
# Table Selector Based on Intent
# -------------------------------
def select_table(intent: str) -> str:
    return {
        "player_week": "player_week",
        "player_season": "players",
        "team_game": "games",
        "team_season": "seasons"
    }.get(intent, "seasons")

# -------------------------------
# Get Table Schema (PRAGMA)
# -------------------------------
def get_table_schema(db_path: str, table_name: str) -> str:
    with sqlite3.connect(db_path) as conn:
        cursor = conn.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        schema_lines = [f"{col[1]} ({col[2]})" for col in columns]
        return "\n".join(schema_lines)

# -------------------------------
# SQL Prompt Builder
# -------------------------------
def build_sql_prompt(question: str, schema: str, table: str, intent: str, entities: dict) -> str:
    season = entities.get("season")
    week = entities.get("week")
    player = entities.get("player")
    opponent = entities.get("opponent")

    # Build structured entity hint for LLM
    entity_block = f"""
EXTRACTED ENTITIES (use ONLY these values):
- season: {season}
- week: {week}
- player: {player}
- opponent: {opponent}
"""

    needs_aggregation = False
    if intent in ("player_week", "player_season"):
        if opponent and not week:
            needs_aggregation = True
        if week is None:
            needs_aggregation = True

    aggregation_rule = ""
    if needs_aggregation and table == "player_week":
        aggregation_rule = (
            "- If multiple rows match (e.g., multiple weeks vs same opponent), "
            "use SUM() for numeric fields.\n"
        )
    
    prompt = f"""
You are a data assistant that writes valid SQLite SQL queries.

TABLE NAME: {table}
SCHEMA:
{schema}

{entity_block}

Rules:
{sql_rules_text}

{aggregation_rule}- Return ONLY the SQL query with no commentary.

User question:
{question}
"""

    print("🟩 SQL Prompt:\n", prompt)
    return prompt


# -------------------------------
# SQL Generation via LLM
# -------------------------------
def generate_sql_query(prompt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=200
    )

    sql = response.choices[0].message.content

    # Clean up markdown-style code block if present
    if sql.startswith("```"):
        sql = re.sub(r"```(sql)?", "", sql).strip("` \n")

    """print("🟦 FINAL SQL QUERY:\n", sql)"""
    return sql

STEP 3: SQL Execution- take the generated SQL and run it against SQLlite db

In [17]:
import sqlite3
import pandas as pd

def run_sql_query(db_path: str, query: str) -> pd.DataFrame:
    """
    Executes a generated SQL query against the SQLite database.
    Returns the results as a pandas DataFrame.
    """
    try:
        with sqlite3.connect(db_path) as conn:
            df = pd.read_sql_query(query, conn)
            return df
    except Exception as e:
        print("❌ SQL EXECUTION ERROR:", e)
        return pd.DataFrame()

STEP 4: AGENTIC REFLECTION TO ASSESS SQL

In [18]:
def reflect_query_results(user_question: str, sql: str, query_results) -> str:
    """
    Reflects on the SQL and results, and determines whether the output answers the user’s question.
    """

    if hasattr(query_results, "to_string"):
        results_str = query_results.to_string(index=False)
    else:
        results_str = str(query_results)

    reflect_prompt = f"""You are a data assistant that evaluates whether a SQL query correctly answered a user's question.

Instructions:
- Look at the original user question.
- Examine the SQL query that was used to answer it.
- Review the returned results.
- Then determine:
  1. Did the SQL query match the user’s intent?
  2. Were the correct filters, fields, and aggregations used?
  3. Were all given rules followed?
  4. Do the results look reasonable?
- If correct, write a brief natural language answer.
- If incorrect, explain what went wrong (e.g. wrong column, filter, aggregation, etc.)

Rules:
    -filtering for team='PHI' is not necessary since all data is for the Eagles.

USER QUESTION:
{user_question}

GENERATED SQL:
{sql}

Rules:
{sql_rules_text}

QUERY RESULTS:
{results_str}
"""

    #print("🧠 Reflection Prompt:\n", reflect_prompt)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": reflect_prompt}],
        temperature=0,
        max_tokens=300
    )

    summary = response.choices[0].message.content.strip()
    return summary

STEP 5: REGENERATE SQL IF NEEDED

In [21]:
def build_regenerated_sql(user_question: str, reflection: str, previous_sql: str, schema: str, table: str, intent: str, entities: dict) -> str:

    regen_prompt = f"""
You are a SQL assistant. Your job is to revise a previous SQL query that failed to correctly answer a user's question.

🟨 Original User Question:
{user_question}

🟥 Identified Problem with Previous SQL:
{reflection}

🧾 Previous SQL:
{previous_sql}

📘 Table Information:
- Table name: {table}
- Intent: {intent}
- Extracted entities: {json.dumps(entities, indent=2)}
- Table schema:
{schema}

Rules:
{sql_rules_text}
🎯 Goal:
Fix the SQL so it accurately answers the user’s question and follows all rules. Only return valid SQLite SQL — no commentary or markdown formatting.
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": regen_prompt}],
        temperature=0,
        max_tokens=200
    )

    new_sql = response.choices[0].message.content

    # Clean up markdown-style code block if present
    if new_sql.startswith("```"):
      new_sql = re.sub(r"```(sql)?", "", new_sql).strip("` \n")

    """print("🟪 REVISED SQL QUERY:\n", new_sql)"""
    return new_sql

6.RUN SQL FUNCTION

In [22]:
def run_sql_query(db_path: str, query: str):
    import sqlite3
    import pandas as pd
    with sqlite3.connect(db_path) as conn:
        return pd.read_sql_query(query, conn)

7. EVALUATE REFLECTION FEEDBACK AND DETERMINE IF REGENERATION IS NECESSARY

In [23]:
def should_regenerate(reflection_summary: str, df_result: pd.DataFrame) -> bool:
    # 1. If empty result, almost always regenerate
    if df_result.empty:
        return True

    # 2. If reflection says it failed
    failure_keywords = [
        "did not successfully answer",
        "did not address",
        "does not address",
        "did not correctly",
        "does not correctly",
        "incorrectly",
        "failed to",
        "no data",
        "returned None",
        "no records found",
        "issue with the query",
        "mismatch",
        "missing"
    ]
    if any(kw in reflection_summary.lower() for kw in failure_keywords):
        return True

    return False


WRAPPED FUNCTION

In [24]:
def run_query_pipeline(question: str):
    from paths import SQLITE_DB_PATH

    # 1️⃣ Intent + Entity Extraction
    parsed = classify_intent_and_extract_entities(question)
    intent = parsed["intent"]
    entities = parsed["entities"]

    # 2️⃣ Table Selection
    table = select_table(intent)

    # 3️⃣ Schema Lookup
    schema = get_table_schema(SQLITE_DB_PATH, table)

    # 4️⃣ SQL Prompt Build
    prompt = build_sql_prompt(question, schema, table, intent, entities)

    # 5️⃣ SQL Generation
    sql = generate_sql_query(prompt)

    # 6️⃣ SQL Execution
    df_result = run_sql_query(SQLITE_DB_PATH, sql)

    # 7️⃣ Reflection
    reflection_summary = reflect_query_results(question, sql, df_result)

    # 8️⃣ Conditional Regeneration
    regenerated_sql = None
    if should_regenerate(reflection_summary, df_result):
        regenerated_sql = build_regenerated_sql(
            question, reflection_summary, sql, schema, table, intent, entities
        )
        df_result = run_sql_query(SQLITE_DB_PATH, regenerated_sql)

    return {
        "question": question,
        "intent": intent,
        "entities": entities,
        "table": table,
        "original_sql": sql,
        "regenerated_sql": regenerated_sql,
        "result": df_result,
        "reflection": reflection_summary
    }

In [25]:
 #Test Questions
#question = "How many wins did the Eagles have against Dallas in 2020?" #agentic needs to assess that teh win coulm is TRUE FALSE text
#question = "How many passing yards did Jalen Hurts have in 2024 against Dallas?"
#question = "How many touchdowns  did the eagles have in 2023 against Dallas?" # pulls 2 rows, good case for agentic refl
#question = "How many yards Jalen Hurts have in 2024 against Dallas?" #pulled only passing, fixed with rule
#question = "What were A.J. Brown's total receiving yards in 2023?"
#question = "How many rushing yards did Miles Sanders have in his career for the eagles"
#question = "Who had the most sacks for the Eagles in 2021?"#agentic refl picked up that the SQL added !=PHI incorrectly, fixed with adjusted rule
#question = "how many wins did the Eagles have over the Cowboys in 2020-2022?"#big fail. Picked season and wins column which doesn't exist
#question = "who was best rusher for the Eagles in 2024 ?"
#question = "What tight end had the most receiving yards for the Eagles in 2023?"# need to map TE to tight_end
#question = "How many receiving yards did Goedert have in 2023?"
#question = "in which game did the eagles take the most sacks?"# big fail and agentic refl caught it. picked wrong turnovers_committed column which doesn't exist and addeed "none" as opponent
#question = "What was eagles red zone touchdown performance in 2023?"
question = "How many rushing yards per game did the eagles have in 2024?"
#question = "in what game did the eagles have the most rushing yards?"#added opponent None
#question = "What Qb was the most accurate for the eagles in 2023?"#Great for agent refl. returned Braden Mann which is correct because his 1 pass was complete. Think about this one. needed to filter to QB
#question = "what was the eagles turnover margin in 2022?"
#question = "how has jalen hurts interception rate improved?"
#question = "how has jalen hurts accuracy improved?"
#question = "how has AJ Brown's yards after catch changed?"#reflection caught that SQL should have included year, fixed with rule
#question = "how did the eagles defense improve in 2024"#tricky one, should have pulled some element of time but pulled the season line



#question= "in which game did the eagles take the most sacks?" #good for agentic refl


output = run_query_pipeline(question)

print("🟨 USER QUESTION:", output["question"])
print("🧠 Intent:", output["intent"])
print("🧩 Entities:", output["entities"])
print("📄 Table:", output["table"])
print("🟦 Original SQL:\n", output["original_sql"])
print("🧠 Reflection:\n", output["reflection"])
print("🔁 Regenerated SQL:\n", output["regenerated_sql"])
print("📊 Result:\n", output["result"])



🟩 SQL Prompt:
 
You are a data assistant that writes valid SQLite SQL queries.

TABLE NAME: seasons
SCHEMA:
season (INTEGER)
week (INTEGER)
league_team_count (INTEGER)
cumulative_points_for_sum (INTEGER)
cumulative_points_against_sum (INTEGER)
cumulative_margin_sum (INTEGER)
cumulative_win_pct (REAL)
cumulative_win_pct_rank_vs_league (INTEGER)
cumulative_win_pct_z_vs_league (REAL)
cumulative_tov_diff (INTEGER)
cumulative_tov_diff_per_game (REAL)
cumulative_tov_diff_per_game_rank_vs_league (INTEGER)
cumulative_tov_diff_per_game_z_vs_league (REAL)
cumulative_yards_gained_sum (INTEGER)
cumulative_yards_allowed_sum (INTEGER)
cumulative_yards_gained_per_game (REAL)
cumulative_yards_allowed_per_game (REAL)
cumulative_yards_gained_per_game_rank_vs_league (INTEGER)
cumulative_yards_gained_per_game_z_vs_league (REAL)
cumulative_yards_allowed_per_game_rank_vs_league (INTEGER)
cumulative_yards_allowed_per_game_z_vs_league (REAL)
cumulative_rush_yards_sum (INTEGER)
cumulative_rush_yards_allowed_su